# Jet Vane Unit Design Document
Design calculations for the hover-jet jet vane actuator unit


In [1]:
import numpy as np
import pint
ureg = pint.UnitRegistry()
print('hello')

hello


## Requirements and variables flow
Required moment on vehicle drives jet vane size
TODO

## Assumptions



In [2]:
r_v = 0.2 * ureg.m   # Vane distance from vehicle center of mass


## Required moment on vehicle
### Required moment on vehicle: angular acceleration
TODO

### Required moment on vehicle: center of mass misalignment
TODO

In [3]:
r_T = 10 * ureg.mm    # Thrust / com misalignment
F_T = 160 * ureg.N    # Thrust force

# Required control force to compensate for com misalignment
F_ctrl = F_T  * r_T / r_v
F_ctrl.ito(ureg.N)
print('Control force F_ctrl = {:~P}'.format(F_ctrl))

Control force F_ctrl = 8.0 N


## Jet flow properties
TODO


In [4]:
T_e = 900 * ureg.K    # Exhaust exit temperature
p_e = 100 * ureg.kPa   # Exhaust exit pressure
R_e = 287 * ureg.J / ureg.kg / ureg.K   # Gas constant
rho_e = p_e / (R_e * T_e)
rho_e.ito(ureg.kg * ureg.m**-3)   # Exhaust density
v_e_full_throttle = 442 * ureg.m / ureg.s   # Exhaust gas velocity at full throttle
gamma = 1.25    # Ratio of specific heats
sonic_e = (gamma * R_e * T_e)**0.5
sonic_e.ito(ureg.m / ureg.s)

print('Exhaust gas density rho_e = {:~P.2}'.format(rho_e))
print('Exit Mach number, full throttle = {:~P.2}'.format(v_e_full_throttle / sonic_e))


Exhaust gas density rho_e = 0.39 kg / m ** 3
Exit Mach number, full throttle = 0.78


## Jet vane size
Let's try to achieve the full desired contorl authority at $1/2$ the full-throttle $v_e$ (roughly 1/4 throttle). Keep the jet vane angle of attack $\alpha$ to be 6 degrees at most.

First, we need a model for how the lift coefficient of the vane varies with angle of attack. The basic model for a thin symmetric airfiol is:

\begin{equation}
C_L = 2 \pi \alpha
\end{equation}

Note that this model is imperfect, the sources of error include:
 - This model assumes a 2-d airfoil, but the vanes will have finite span (and low aspect ratio) so there will be 3d flow effects.
 - This model assumes incompressible flow, but the exhaust jet will have a high Mach number (~0.8 at full throttle) so there will be compressibility effects.
 - There will be flow interactions between:
   * a vane and the wall the vane is mounted through (this will "mirror" the flow and reduce the vane's low asptect ratio penalty)
   * a vane and the other vanes (this will induce weird couplings and non-linearities)
 - The vanes will not operate in a clean flow. There will probably be some swirl and velocity non-uniformity in the jet exhaust. The amout of swirl may be different at different throttle settings.
 - This model assumes steady flow - it won't be valid if the vanes are moving rapidly
There are correction formulas for some of these effects in certain circumstances, but I don't think they are worth taking into account here. We should just press forward with the simple model and realize it's rough.

The required vane force is

\begin{equation}
\frac{F_{ctrl}}{2} = F_v = \frac{1}{2} \rho_e v_e^2 C_L S_v
\end{equation}

where $S_v$ is the vane reference area and the lefthand side is divided by 2 because there are 2 vanes per axis.


In [5]:
# Throttle setting adjustment
throttle = 0.25   # Throttle setting at which to size vanes
# rough approx. - changing throttle would also change exhuast gas temperature and thus density
v_e = v_e_full_throttle * (throttle)**0.5   # Exit velocity at throttle setting
M_e = v_e / sonic_e   # Exit mach number at throttle setting
F_ctrl = F_T * throttle * r_T / r_v  # required vane force due to cm misalign


A_R = 1.33   # Jet vane aspect ratio
alpha = np.deg2rad(6)

# jet vane lift coeff
C_L = 2 * np.pi * alpha
print('Jet vane lift coeff C_L = {:.2f}'.format(C_L))

# Jet vane effective area - minimum viable area for control
S_v_min = F_ctrl / (rho_e * v_e**2 * C_L)
S_v_min.ito(ureg.m**2)
print('Jet vane effective area, minimum for control S_v_min = {:~P.2}'.format(S_v_min))

# jet vane span
b_v = (A_R * S_v_min)**0.5
b_v.ito(ureg.m)
print('Jet vane span b_v = {:~P.3}'.format(b_v.to(ureg.mm)))

# Jet vane span sanity check
b_v_max = 20 * ureg.mm   # Max possible vane (effective) span, a little less than the engine nozzle exit radius
if b_v > b_v_max:
    print('Vane span is too big!')

Jet vane lift coeff C_L = 0.66
Jet vane effective area, minimum for control S_v_min = 0.00016 m ** 2
Jet vane span b_v = 14.6 mm


Let's be conservative and use a larger vane, with a 20 x 15 mm effective area in the flow:

In [6]:
b_v = 20 * ureg.mm    # Vane span
c_v = 20 * ureg.mm    # Vane chord
S_v = b_v * c_v    # Vane effective area
S_v.ito(ureg.m**2)
print('Vane area margin S_v / S_v_min = {:.2f}'.format(S_v / S_v_min))

Vane area margin S_v / S_v_min = 2.49 dimensionless


## Jet vane timescale
When the angle of attack of an airfoil is changed, it takes some time for the flow to adjust. For our lift model to be valid, the timescale on which we control the vane rotation should be longer than the timescale for flow over the vane. The vane flow timescale is roughly:

\begin{equation}
\tau_{flow} = \frac{c_v}{v_e}
\end{equation}

The following calculation shows that the vane timescale is much faster than our control loop, so we don't need to worry about this.

In [7]:
# Vane flow timescale
flow_timescale = c_v / v_e
flow_timescale.ito(ureg.ms)
print('Vane flow timescale = {:~P.2}'.format(flow_timescale))

Vane flow timescale = 0.09 ms


## Jet vane drag
The drag coefficient is predicted to be:

\begin{equation}
C_D = C_{D0} + k C_L^2 = C_{D0} + 4 \pi^2 k \alpha^2
\end{equation}

According to lifting line theory, $k = 1 / (\pi A_R)$ - for our purposes let's assume $k = 1/3$. $C_{D0}$ is the minimum/profile drag coefficient - let's assume $C_{D0} = 0.05$. We will also include a Prandtl-Glauert correction term to account for the increase of drag at high mach numbers:

\begin{equation}
C_D = C_{D0} + \frac{4 \pi^2 k \alpha^2}{\sqrt{1 - M_e^2}}
\end{equation}


In [8]:
# Conditions at which to compute vane drag
v_e = v_e_full_throttle
M_e = v_e / sonic_e
alpha = np.deg2rad(6)    # vane angle of attack

# Drag model parameters
C_D0 = 0.05    # Minimum drag coeff
k = 1/3.    # Induced drag coefficient

C_L = 2 * np.pi * alpha
C_D = C_D0 + (k * C_L**2) / (1 - M_e**2)**0.5

L = 0.5 * rho_e * v_e**2 * C_L * S_v
D = 0.5 * rho_e * v_e**2 * C_D * S_v
L.ito(ureg.N)
D.ito(ureg.N)
print('Vane lift force L = {:~P.2}'.format(L))
print('Vane drag force D = {:~P.2}'.format(D))
print('Vane L/D = {:.2f}'.format(L/D))

Vane lift force L = 1e+01 N
Vane drag force D = 4.2 N
Vane L/D = 2.35 dimensionless


## Jet vane actuator torque
The jet vane actuator must supply enough torque to overcome 1) the aerodynamic moment on the vane $M_a$ and 2) the friction moment from the bearing $M_f$. Thus:

\begin{equation}
\tau_{act} = M_a + M_f
\end{equation}

The aerodyamic moment can be reduced by mounting the vane at its center of pressure, about which their should be no aerodynamic moment. For thin symmetric airfoils, this is $c/4$ from the leading edge; this point is also the aerodynamic center so the moment should not change with angle of attack. Practically, there may still be some moment due to swirl in the flow, weirdness at high Mach number, interaction effects, etc. To account for this, we will assume a moment coefficient of $C_M = 0.1$.

\begin{equation}
M_a = \frac{1}{2} \rho_e v_e^2 C_M c_v^2 b_v
\end{equation}

the bearing friction moment is
\begin{equation}
M_f = \mu r_{shaft} F_v
\end{equation}

where $\mu$ is the bearing/shaft friction coefficient, $r_{shaft}$ is the shaft radius and $F_v$ is the side load on the shaft.

In [9]:
C_M = 0.1   # Aerodynamic moment coefficient
mu_bearing = 0.3    # Bearing friciton coefficient
r_shaft = 1/8. * ureg.inch   # Shaft radius
F_v = (L**2 + D**2)**0.5   # Shaft load

# Aerodynamic moment
M_a = 0.5 * rho_e * v_e**2 * C_M * c_v**2 * b_v
M_a.ito(ureg.N * ureg.m)
print('Aero moment M_a = {:~P.2}'.format(M_a))

# Bearing friction moment
M_f = mu_bearing * r_shaft * F_v
M_f.ito(ureg.N * ureg.m)
print('Friction moment M_f = {:~P.2}'.format(M_f))

# Torque margin
torque_margin = 4.
motor_torque = torque_margin * (M_a + M_f)
print('Min. acceptable motor toque > {:~P.2} ({:~P.3})'.format(motor_torque, motor_torque.to(ureg.ozf * ureg.inch)))

Aero moment M_a = 0.03 N * m
Friction moment M_f = 0.01 N * m
Min. acceptable motor toque > 0.16 N * m (23.0 in * ozf)
